In [1]:
import pandas as pd
import sqlalchemy
#from sqlalchemy import create_engine
from sqlalchemy.engine import create_engine


In [8]:
purchase_events_df = spark.read.parquet("/tmp/purchase_events")

In [10]:
purchase_events_df.show(50,truncate=False)

+-----------------------+----------+--------------------+--------------+---------+--------+-----------+-----+
|event_ts               |userid    |Host                |event_type    |name     |strength|n_purchased|price|
+-----------------------+----------+--------------------+--------------+---------+--------+-----------+-----+
|2021-12-05 01:36:20.454|'user-002'|user-002.comcast.com|purchase_sword|excalibur|1000    |9          |2000 |
|2021-12-05 01:36:25.995|'user-008'|user-008.comcast.com|purchase_sword|excalibur|1000    |7          |2000 |
|2021-12-05 01:36:20.429|'user-002'|user-002.comcast.com|purchase_sword|excalibur|1000    |9          |2000 |
|2021-12-05 01:36:24.537|'user-006'|user-006.comcast.com|purchase_sword|excalibur|1000    |2          |2000 |
|2021-12-05 01:36:23.88 |'user-004'|user-004.comcast.com|purchase_knife|kukri    |500     |7          |1000 |
|2021-12-05 01:36:24.543|'user-006'|user-006.comcast.com|purchase_sword|excalibur|1000    |2          |2000 |
|2021-12-0

In [4]:
guid_events_df = spark.read.parquet("/tmp/guild_events")

In [5]:
guid_events_df.show(10,truncate=False)

+-----------------------+----------+--------------------+----------+---------------+--------+-----------+-----+
|event_ts               |userid    |Host                |event_type|name           |strength|n_purchased|price|
+-----------------------+----------+--------------------+----------+---------------+--------+-----------+-----+
|2021-12-05 01:36:23.359|'user-002'|user-002.comcast.com|join_guild|Game of Thrones|1200    |1          |1000 |
|2021-12-05 01:36:23.371|'user-002'|user-002.comcast.com|join_guild|Game of Thrones|1200    |1          |1000 |
|2021-12-05 01:36:23.364|'user-002'|user-002.comcast.com|join_guild|Game of Thrones|1200    |1          |1000 |
|2021-12-05 01:36:23.376|'user-002'|user-002.comcast.com|join_guild|Game of Thrones|1200    |1          |1000 |
|2021-12-05 01:36:21.812|'user-004'|user-004.comcast.com|join_guild|Castle of Rock |1200    |1          |1000 |
|2021-12-05 01:36:21.809|'user-004'|user-004.comcast.com|join_guild|Castle of Rock |1200    |1          

In [7]:
purchase_events_df.registerTempTable('purchase_events')

In [8]:
spark.sql("""
            select *
            from purchase_events
            Limit 1
          """).show(truncate=False)

+-----------------------+----------+--------------+--------------+---------+--------+-----------+-----+
|event_ts               |userid    |Host          |event_type    |name     |strength|n_purchased|price|
+-----------------------+----------+--------------+--------------+---------+--------+-----------+-----+
|2021-12-04 01:42:35.954|'user-007'|localhost:5000|purchase_sword|excalibur|1000    |6          |2000 |
+-----------------------+----------+--------------+--------------+---------+--------+-----------+-----+



In [15]:
purchase_events_df.printSchema()

root
 |-- event_ts: timestamp (nullable = true)
 |-- userid: string (nullable = true)
 |-- Host: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- strength: long (nullable = true)
 |-- n_purchased: long (nullable = true)
 |-- price: long (nullable = true)



In [16]:
purchase_events_df.columns

['event_ts',
 'userid',
 'Host',
 'event_type',
 'name',
 'strength',
 'n_purchased',
 'price']

In [17]:
guild_events_df = spark.read.parquet("/tmp/guild_events")

In [18]:
guild_events_df.registerTempTable('guild_events')

In [19]:
guild_events_df.printSchema()

root
 |-- event_ts: timestamp (nullable = true)
 |-- userid: string (nullable = true)
 |-- Host: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- strength: long (nullable = true)
 |-- n_purchased: long (nullable = true)
 |-- price: long (nullable = true)



In [22]:
spark.catalog.listTables()

[Table(name='guild_events', database='default', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='purchase_events', database='default', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='guild_events', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='purchase_events', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [20]:
q1_query = """
               SELECT event_type,
                   sum(amount_purchased) as ttl_purchased
               FROM   purchase_events
               GROUP  BY event_type 
           """

In [21]:
spark.sql(q1_query).show(truncate=False)

AnalysisException: "cannot resolve '`amount_purchased`' given input columns: [price, strength, n_purchased, name, event_type, userid, Host, event_ts]; line 3 pos 23;\n'Aggregate [event_type#75], [event_type#75, 'sum('amount_purchased) AS ttl_purchased#157]\n+- SubqueryAlias purchase_events\n   +- Relation[event_ts#72,userid#73,Host#74,event_type#75,name#76,strength#77L,n_purchased#78L,price#79L] parquet\n"